In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from itertools import combinations
from uuid import uuid4

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels import api as sm

import bhutils
from api.blpw import BlpQuery
from api.history_plotter import get_expressions_data
from api.tools import get_history_plotter_strings, parse_swap_periods, parse_clipboard, parse_curve
from api.utils import parse_offset
from api.mappings import strategy_map

pd.options.plotting.backend = "plotly"

In [3]:
y_col = 'y'
whole_history = '10y'

In [5]:
r = get_history_plotter_strings(
    # delta_mul=-1, 
    delta_tol=499
)
expressions = r[-1].to_dict()

C:\dev\projects\menashe\api\tools.py:204: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dg_ = dg.groupby(['Strategy', 'string', 'string_aged'], as_index=False).sum()


In [35]:
future_expiry_month_codes = {
    1: 'F',
    2: 'G',
    3: 'H',
    4: 'J',
    5: 'K',
    6: 'M',
    7: 'N',
    8: 'Q',
    9: 'U',
    10: 'V',
    11: 'X',
    12: 'Z',
    'F': 1,
    'G': 2,
    'H': 3,
    'J': 4,
    'K': 5,
    'M': 6,
    'N': 7,
    'Q': 8,
    'U': 9,
    'V': 10,
    'X': 11,
    'Z': 12
}

In [38]:
def get_previous_monday(date):
    return date if not date.weekday() else date - pd.tseries.offsets.DateOffset(days=date.weekday())


def get_expiry(n=1, date=None, mult=3, settle=False):
    date = date or pd.Timestamp.today().floor('d')
    month = date.month
    temp = mult * (month // mult + 1) if month % mult else month

    temp_expiry = pd.Timestamp(year=date.year, month=temp, day=19)
    expiry = get_previous_monday(temp_expiry)

    if date >= expiry:
        expiry = get_previous_monday(expiry + pd.tseries.offsets.DateOffset(months=3))
    else:
        expiry = expiry

    if n > 1:
        expiry = get_previous_monday(expiry + pd.tseries.offsets.DateOffset(months=3*(n-1)))
    if settle:
        expiry = expiry + pd.tseries.offsets.BDay(2)
    
    return expiry

def get_expiry_from_code(code, date=None):
    date = date or pd.Timestamp.today().floor('d')
    m, y = code[0], code[1]
    year = date.year
    yr_str = str(year)

    if int(yr_str[-1]) <= int(y):
        y_ = f"{yr_str[:-1]}{y}"
    else:
        y_ = f"{yr_str[:2]}{int(yr_str[2]+1)}{y}"

    y_ = int(y_)
    return get_previous_monday(pd.Timestamp(year=y_, month=future_expiry_month_codes.get(m), day=19))

In [14]:
code = 'H5'
date = None 


In [ ]:
def get_future_number(code, date=None):
    

In [ ]:
ns = range(1, 8 + 1)
start_dates = [get_expiry(n, settle=True) for n in ns]
expiry_tenors = [parse_swap_periods(start_date=start_date, end_date='3m') for start_date in start_dates]

In [ ]:
curves = {
    'er': 'eur.3m', 
    'ed': 'usd.3m'
}

In [ ]:
X_expressions = {}
for name, curve in curves.items():
    for i, t in enumerate(expiry_tenors, 1):
        X_expressions[f"{name}{i}"] = f"[s {curve} {t}]"   

In [ ]:
X_data = get_expressions_data(X_expressions, history=whole_history)

In [ ]:
subsets = [
    [*X_data[X_expressions.keys()].columns[[*comb]]] 
    for comb in combinations(range(X_data[X_expressions.keys()].shape[1]), 3)
]

In [ ]:
def get_df_reg(X, y, diff=False, history=None):
    if diff:
        X = X.diff().dropna()
        y = y.diff().dropna()
    if history:
        X = X.loc[X.index[-1] - parse_offset(history):, :]
        y = y.loc[y.index[-1] - parse_offset(history):]
    df_reg = pd.concat([X, y], axis=1).ffill()
    return df_reg

In [ ]:
def filter_high_corr(df, threshold=0.6, y_col='y'):
    corr = df.corr()

    melt_corr = corr.drop(y_col, axis=1).drop(y_col).melt(ignore_index=False).reset_index()
    melt_corr = melt_corr.loc[melt_corr['variable'] != melt_corr['index']]
    

    indices = []
    len_corr = len(corr) - 1  # excluding y 
    for i in range(len_corr):
        l = len_corr - 1
        start = i * l + i
        end = i * l + l
        indices.extend([*range(start, end)])
    unique_corr = melt_corr.iloc[indices]

    filtered_corr = unique_corr.loc[unique_corr['value'].abs() > threshold]
    drop = []

    for index, row in filtered_corr.iterrows():
        if abs(corr.loc[y_col, row['index']]) > abs(corr.loc[y_col, row['variable']]):
            drop.append(row['variable'])
        else:
            drop.append(row['index'])
    
    return df.drop(drop, axis=1)

In [ ]:
def run_regressions(
    X,
    y,
    subsets,  
    history='2y',
    diff=True,
    corr_threshold=None,
    y_col=None,
    sort=True
):
    lms_index = []
    lms_data = []
    lms = {}
    olss = {}
    ols_res = {}
    for subset in subsets:

        df_reg = get_df_reg(
            X[subset], 
            y,
            diff=diff,
            history=history
        )

        y_col = y_col or df_reg.columns[-1]

        if corr_threshold:
            df_reg1 = filter_high_corr(df_reg, threshold=corr_threshold)
        else:
            df_reg1 = df_reg

        key = tuple([*df_reg1.iloc[:, :-1].columns])

        if key not in lms:
            lm = LinearRegression(fit_intercept=True)
            lm.fit(df_reg1.iloc[:, :-1], df_reg1.iloc[:, -1])
            lms[key] = lm

            ols = sm.OLS(df_reg1.iloc[:, -1], sm.add_constant(df_reg1.iloc[:, :-1]))
            res = ols.fit()

            olss[key] = ols
            ols_res[key] = res

            lms_data.append({
                'params': np.round(lm.coef_, 2), 
                'intercept':  np.round(lm.intercept_, 2),
                'r2':  np.round(lm.score(df_reg1.iloc[:, :-1], df_reg1.iloc[:, -1]), 4),
                'contracts': - np.round((lm.coef_ / 2500) * 1e6).astype(int)
            })
    
    summary = pd.DataFrame(lms_data, index=[*lms])
    return {
        'summary': summary.sort_values('r2', ascending=False) if sort else summary,
        'lms': lms,
        'ols': olss,
        'ols_res': ols_res
    }

In [ ]:
corr_threshold = 0.7
histories = ['2y', '1y', '6m']

In [ ]:
ress = {}
summary = {}
for hist in histories:
    res = run_regressions(
        X=X_data,
        y=y,
        subsets=subsets,
        corr_threshold=corr_threshold,
        history=hist
    )
    ress[hist] = res
    summary[hist] = res['summary']
summary_df = pd.concat(summary)

In [ ]:
ss = summary_df\
        .rename_axis(['history', 'futures']).reset_index()\
        .pivot(index='futures', columns='history')\
        .reorder_levels([1, 0], axis=1).loc[:, histories]
ss = ss.assign(r2_avg=ss.loc[:, (slice(None),'r2')].mean(axis=1).round(4))\
       .sort_values('r2_avg', ascending=False)\
       .reset_index()

In [ ]:
ss.to_excel(f'reg{pd.Timestamp.today():%Y%m%d}.xlsx')

In [ ]:
ss

### Hedged Portfolio

In [ ]:
n = 3
h = '6m'

In [ ]:
def get_summary_df(df):
    stats = {
        'last': df.iloc[-1],
        'first': df.iloc[0],
        'min': df.min(),
        'max': df.max(),
        'mean': df.mean(),
        'std': df.std(),
        'zscore': (df.iloc[-1] - df.mean()) / df.std(),
        'q1': df.quantile(0.25),
        'median': df.quantile(0.5),
        'q3': df.quantile(0.75),
    }
    
    stats_df = pd.concat(stats).to_frame()\
                                .reset_index()\
                                .pivot(columns='level_0', index='level_1')\
                                .droplevel(0, axis=1)\
                                .rename_axis('stats', axis=1)\
                                .rename_axis('series')
    
    return stats_df.loc[df.columns, [*stats]]

In [ ]:
def get_hedge(summary_df, mapping, n=0, h='6m', c=-1):
    return ''.join([f"{p*c:+}*{X_expressions[f]}" for f, p in zip(ss.loc[n, 'futures'].squeeze(), ss.loc[n, (h, 'params')])])

In [ ]:
new_e = {
    **expressions,
    'hedge': get_hedge(ss, X_expressions, n=n),
    'ptf_hedge': 'ptf + hedge'
}

In [ ]:
new_data = get_expressions_data(new_e, history='6m')

In [ ]:
(-new_data[['ptf', 'hedge', 'ptf_hedge']]).plot(height=700)

In [ ]:
get_summary_df(new_data[['ptf', 'hedge', 'ptf_hedge']].diff().dropna()).round(4)